In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import os
import pandas as pd
#Functional coding
import functools
from functools import partial
from tensorflow.python.ops import array_ops 

In [2]:
#Data Path..
Datapath="DATA/Reservoir/Reservoir_Data.txt"
Labelpath="DATA/Reservoir/Reservoir_Label.txt"
Rewardpath="DATA/Reservoir/Reservoir_Reward.txt"

In [3]:
#Given local path, find full path
def PathFinder(path):
    #python 2
    #script_dir = os.path.dirname('__file__')
    #fullpath = os.path.join(script_dir,path)
    #python 3
    fullpath=os.path.abspath(path)
    print(fullpath)
    return fullpath

#Read Data for Deep Learning
def ReadData(path):
    fullpath=PathFinder(path)
    return pd.read_csv(fullpath, sep=',', header=0)

In [4]:
S_A_pd = ReadData(Datapath)
SP_pd = ReadData(Labelpath)
R_pd = ReadData(Rewardpath)
S_A_matrix=S_A_pd.as_matrix()
SP_matrix=SP_pd.as_matrix()
R_matrix=R_pd.as_matrix()

/home/wuga/Documents/Notebook/VAE-PLANNING/DATA/Reservoir/Reservoir_Data.txt
/home/wuga/Documents/Notebook/VAE-PLANNING/DATA/Reservoir/Reservoir_Label.txt
/home/wuga/Documents/Notebook/VAE-PLANNING/DATA/Reservoir/Reservoir_Reward.txt


In [5]:
default_settings = {
    "max_cap"          : [100, 200, 400, 500],
    "high_bound"       : [80, 180, 380, 480],
    "low_bound"        : [20, 30, 40, 60],
    "rain"             : [5, 10, 20, 30],
    "downstream"       : [[1,2],[2,3],[3,4]],
    "downtosea"        : [4],
    "biggestmaxcap"    : 1000,
    "reservoirs"    : [1,2,3,4],
    "init_state"       : [75,50,50,50]
   }


In [6]:
class RESERVOIR(object):
    def __init__(self, 
                 batch_size,
                 default_settings):
        self.batch_size = batch_size
        self.reservoirs = default_settings['reservoirs']
        self.reservoir_num = len(default_settings['reservoirs'])
        self.biggestmaxcap = tf.constant(default_settings["biggestmaxcap"],dtype=tf.float32)
        self.zero = tf.constant(0,shape=[self.batch_size,self.reservoir_num],dtype=tf.float32)
        self._high_bounds(default_settings["high_bound"])
        self._low_bounds(default_settings["low_bound"])
        self._rains(default_settings["rain"])
        self._max_cap(default_settings["max_cap"])
        self._downstream(default_settings["downstream"])
        self._downtosea(default_settings["downtosea"])
        
    def _max_cap(self, max_cap_list):
        self.max_cap = tf.constant(max_cap_list,dtype=tf.float32)
    
    def _high_bounds(self, high_bound_list):
        self.high_bound = tf.constant(high_bound_list,dtype=tf.float32)
            
    def _low_bounds(self, low_bound_list):
        self.low_bound = tf.constant(low_bound_list,dtype=tf.float32)
            
    def _rains(self, rain_list):
        self.rain = tf.constant(rain_list,dtype=tf.float32)
        
    def _downstream(self, downstream):
        np_downstream = np.zeros((self.reservoir_num,self.reservoir_num))
        for i in downstream:
            m = self.reservoirs.index(i[0])
            n = self.reservoirs.index(i[1])
            np_downstream[m,n] = 1
        self.downstream = tf.constant(np_downstream,dtype=tf.float32)
        
    def _downtosea(self, downtosea):
        np_downtosea = np.zeros((self.reservoir_num,))
        for i in downtosea:
            m = self.reservoirs.index(i)
            np_downtosea[m] = 1
        self.downtosea =  tf.constant(np_downtosea,dtype=tf.float32)
            
    def MAXCAP(self):
        return self.max_cap
    
    def HIGH_BOUND(self):
        return self.high_bound
    
    def LOW_BOUND(self):
        return self.low_bound
    
    def RAIN(self):
        return self.rain
    
    def DOWNSTREAM(self):
        return self.downstream
    
    def DOWNTOSEA(self):
        return self.downtosea
        
    def BIGGESTMAXCAP(self):
        return self.biggestmaxcap
    
    def Transition(self, states, actions):
        previous_state = states
        vaporated = 0.5*tf.sin(previous_state/self.BIGGESTMAXCAP())*previous_state
        upstreamflow = tf.transpose(tf.matmul(tf.transpose(self.DOWNSTREAM()),tf.transpose(actions)))
        new_state = previous_state + self.RAIN()-vaporated-actions+ upstreamflow                        
        return new_state
    
    #Reward for Reservoir is computed on 'Next State'
    def Reward(self, states):
        new_rewards = tf.select(tf.logical_and(tf.greater_equal(states,self.LOW_BOUND()),tf.less_equal(states,self.HIGH_BOUND())),\
                                 self.zero,\
                                tf.select(tf.less(states,self.LOW_BOUND()),\
                                          -5*(self.LOW_BOUND()-states),\
                                         -100*(states-self.HIGH_BOUND()))\
                               )
        new_rewards+=tf.abs(((self.HIGH_BOUND()+self.LOW_BOUND())/2.0)-states)*(-0.1)
        return tf.reduce_sum(new_rewards,1,keep_dims=True)       

In [7]:
# States
states = tf.placeholder(tf.float32,[10, 4],name="States")

# Actions
actions = tf.placeholder(tf.float32,[10, 4],name="Actions")

In [8]:
reservoir_inst = RESERVOIR(10, default_settings)

In [9]:
states_list=tf.unpack(states)
actions_list = tf.unpack(actions)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
feed_dict={states:S_A_matrix[:10,4:], actions:S_A_matrix[:10,:4]}
new_state = reservoir_inst.Transition(states,actions)
print(sess.run([new_state], feed_dict=feed_dict))

[array([[  34.2109375 ,   98.94076538,   70.70948792,   79.58050537],
       [  29.57379913,  105.29424286,   68.97597504,  121.5150528 ],
       [  30.76093674,   61.79491806,  125.72145844,  117.12879944],
       [  31.53264618,   47.34428787,  159.31442261,  142.84954834],
       [  34.37532425,   53.70756912,  153.4153595 ,  173.37290955],
       [  29.43461418,   63.54294586,  123.92326355,  192.53741455],
       [  25.87185097,   59.27878571,  152.45231628,  135.31620789],
       [  24.97044945,   56.14728165,  131.0018158 ,  143.20846558],
       [  26.93314552,   61.22503662,   68.16318512,  211.58776855],
       [  29.30649757,   50.83280182,  100.35129547,  114.01165771]], dtype=float32)]


In [10]:
new_rewards = reservoir_inst.Reward(states)

In [11]:
feed_dict={states:S_A_matrix[:10,4:], actions:S_A_matrix[:10,:4]}
sess.run(new_rewards,feed_dict=feed_dict )

array([[-96.        ],
       [-35.15583038],
       [-31.02294159],
       [-29.95938873],
       [-25.39591026],
       [-22.01288414],
       [-22.55617905],
       [-26.20808601],
       [-27.96719742],
       [-26.70908737]], dtype=float32)

In [12]:
class RESERVOIRCell(tf.nn.rnn_cell.RNNCell):

    def __init__(self, batch_size,default_settings):
        self._num_state_units = len(default_settings["reservoirs"])
        self._num_reward_units = self._num_state_units +1
        self.reservoir = RESERVOIR(batch_size,default_settings)

    @property
    def state_size(self):
        return self._num_state_units

    @property
    def output_size(self):
        return self._num_reward_units

    def __call__(self, inputs, state, scope=None):
        next_state =  self.reservoir.Transition(state, inputs)
        reward = self.reservoir.Reward(next_state)   
        return tf.concat(1,[reward,next_state]), next_state

In [13]:
class ActionOptimizer(object):
    def __init__(self,
                a, # Actions
                num_step, # Number of RNN step, this is a fixed step RNN sequence, 12 for navigation
                num_act, # Number of actions
                batch_size, #Batch Size
                learning_rate=0.1): 
        self.action = a
        print(self.action)
        self.batch_size = batch_size
        self.num_step = num_step
        self.learning_rate = learning_rate
        self._p_create_rnn_graph()
        self._p_Q_loss()
        self.sess = tf.InteractiveSession()
        self.sess.run(tf.global_variables_initializer())
    
    def _p_create_rnn_graph(self):
        cell = RESERVOIRCell(self.batch_size, default_settings)
        initial_state = cell.zero_state(self.batch_size, dtype=tf.float32)+tf.constant([default_settings["init_state"]],dtype=tf.float32)
        print('action batch size:{0}'.format(array_ops.shape(self.action)[0]))
        print('Initial_state shape:{0}'.format(initial_state))
        rnn_outputs, state = tf.nn.dynamic_rnn(cell, self.action, dtype=tf.float32,initial_state=initial_state)
        #need output intermediate states as well
        concated = tf.concat(0,rnn_outputs)
        print('concated shape:{0}'.format(concated.get_shape()))
        something_unpacked =  tf.unpack(concated, axis=2)
        self.outputs = tf.reshape(something_unpacked[0],[-1,self.num_step,1])
        print(' self.outputs:{0}'.format(self.outputs.get_shape()))
        self.intern_states = tf.pack([something_unpacked[i+1] for i in range(len(default_settings["reservoirs"]))], axis=2)
        self.last_state = state
        self.pred = tf.reduce_sum(self.outputs,1)
        self.average_pred = tf.reduce_mean(self.pred)
        print("self.pred:{0}".format(self.pred))
            
    def _p_create_loss(self):

        objective = tf.reduce_mean(self.pred) 
        self.loss = -objective
        print(self.loss.get_shape())
        #self.loss = -objective
        self.optimizer = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss, var_list=[a])
        
    def _p_Q_loss(self):
        objective = tf.constant(0.0, shape=[self.batch_size, 1])
        for i in range(self.num_step):
            Rt = self.outputs[:,i]
            SumRj=tf.constant(0.0, shape=[self.batch_size, 1])
            #SumRk=tf.constant(0.0, shape=[self.batch_size, 1])
            if i<(self.num_step-1):
                j = i+1
                SumRj = tf.reduce_sum(self.outputs[:,j:],1)
            #if i<(self.num_step-1):
                #k= i+1
                #SumRk = tf.reduce_sum(self.outputs[:,k:],1)
            objective+=(Rt*SumRj+tf.square(Rt))*(self.num_step-i)/np.square(self.num_step)
        self.loss = tf.log(tf.reduce_mean(tf.square(objective)))
        self.optimizer = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss, var_list=[a])
        
    def Optimize(self,epoch=100):
        
        new_loss = self.sess.run([self.average_pred])
        print('Loss in epoch {0}: {1}'.format("Initial", new_loss)) 
        for epoch in range(epoch):
            training = self.sess.run([self.optimizer])
            action_upperbound=self.sess.run(self.intern_states)
            self.sess.run(tf.assign(self.action, tf.clip_by_value(self.action, 0, action_upperbound)))
            if True:
                new_loss = self.sess.run([self.average_pred])
                print('Loss in epoch {0}: {1}'.format(epoch, new_loss))  
        minimum_costs_id=self.sess.run(tf.argmax(self.pred,0))
        print(minimum_costs_id)
        best_action = self.sess.run(self.action)[minimum_costs_id[0]]
        print('Optimal Action Squence:{0}'.format(best_action))
        pred_list = self.sess.run(self.pred)
        pred_list=np.sort(pred_list.flatten())[::-1]
        pred_list=pred_list[:5]
        pred_mean = np.mean(pred_list)
        pred_std = np.std(pred_list)
        print('Best Cost: {0}'.format(pred_list[0]))
        print('Sorted Costs:{0}'.format(pred_list))
        print('MEAN: {0}, STD:{1}'.format(pred_mean,pred_std))
        print('The last state:{0}'.format(self.sess.run(self.last_state)))
        print('The last state:{0}'.format(self.sess.run(self.last_state)[minimum_costs_id[0]]))
        print('Rewards each time step:{0}'.format(self.sess.run(self.outputs)[minimum_costs_id[0]]))
        print('Intermediate states:{0}'.format(self.sess.run(self.intern_states)[minimum_costs_id[0]]))

In [14]:
a = tf.Variable(tf.truncated_normal(shape=[100,120,4],mean=0.0, stddev=0.5),name="action")
rnn_inst = ActionOptimizer(a, 120,4,100)  

Tensor("action/read:0", shape=(100, 120, 4), dtype=float32)
action batch size:Tensor("strided_slice:0", shape=(), dtype=int32)
Initial_state shape:Tensor("add_4:0", shape=(100, 4), dtype=float32)
concated shape:(100, 120, 5)
 self.outputs:(100, 120, 1)
self.pred:Tensor("Sum_1:0", shape=(100, 1), dtype=float32)


In [15]:
rnn_inst.Optimize(1000)

Loss in epoch Initial: [-218543.2]
Loss in epoch 0: [-197794.36]
Loss in epoch 1: [-197791.73]
Loss in epoch 2: [-197789.0]
Loss in epoch 3: [-197786.09]
Loss in epoch 4: [-197783.06]
Loss in epoch 5: [-197779.84]
Loss in epoch 6: [-197776.45]
Loss in epoch 7: [-197772.88]
Loss in epoch 8: [-197769.08]
Loss in epoch 9: [-197765.12]
Loss in epoch 10: [-197760.94]
Loss in epoch 11: [-197756.52]
Loss in epoch 12: [-197751.88]
Loss in epoch 13: [-197746.95]
Loss in epoch 14: [-197741.8]
Loss in epoch 15: [-197736.34]
Loss in epoch 16: [-197730.59]
Loss in epoch 17: [-197724.55]
Loss in epoch 18: [-197718.14]
Loss in epoch 19: [-197711.42]
Loss in epoch 20: [-197704.34]
Loss in epoch 21: [-197696.84]
Loss in epoch 22: [-197688.95]
Loss in epoch 23: [-197680.64]
Loss in epoch 24: [-197671.88]
Loss in epoch 25: [-197662.64]
Loss in epoch 26: [-197652.91]
Loss in epoch 27: [-197642.66]
Loss in epoch 28: [-197631.81]
Loss in epoch 29: [-197620.41]
Loss in epoch 30: [-197608.41]
Loss in epoch 31